In [30]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, sum, avg, max, min

spark = SparkSession.Builder().getOrCreate()

In [3]:
simpleData = [("James","Sales","NY",90000,34,10000),
    ("Michael","Sales","NY",86000,56,20000),
    ("Robert","Sales","CA",81000,30,23000),
    ("Maria","Finance","CA",90000,24,23000),
    ("Raman","Finance","CA",99000,40,24000),
    ("Scott","Finance","NY",83000,36,19000),
    ("Jen","Finance","NY",79000,53,15000),
    ("Jeff","Marketing","CA",80000,25,18000),
    ("Kumar","Marketing","NY",91000,50,21000)
  ]

schema = ["employee_name","department","state","salary","age","bonus"]
df = spark.createDataFrame(data=simpleData, schema = schema)

In [5]:
df.show()

+-------------+----------+-----+------+---+-----+
|employee_name|department|state|salary|age|bonus|
+-------------+----------+-----+------+---+-----+
|        James|     Sales|   NY| 90000| 34|10000|
|      Michael|     Sales|   NY| 86000| 56|20000|
|       Robert|     Sales|   CA| 81000| 30|23000|
|        Maria|   Finance|   CA| 90000| 24|23000|
|        Raman|   Finance|   CA| 99000| 40|24000|
|        Scott|   Finance|   NY| 83000| 36|19000|
|          Jen|   Finance|   NY| 79000| 53|15000|
|         Jeff| Marketing|   CA| 80000| 25|18000|
|        Kumar| Marketing|   NY| 91000| 50|21000|
+-------------+----------+-----+------+---+-----+



In [11]:
df.groupBy("department").sum("salary").show(truncate=False)

+----------+-----------+
|department|sum(salary)|
+----------+-----------+
|Sales     |257000     |
|Finance   |351000     |
|Marketing |171000     |
+----------+-----------+



In [12]:
# count employee in each department
df.groupBy("department").count().show()

+----------+-----+
|department|count|
+----------+-----+
|     Sales|    3|
|   Finance|    4|
| Marketing|    2|
+----------+-----+



In [15]:
# calculate min salary in each department
df.groupBy("department").min("salary").show()

+----------+-----------+
|department|min(salary)|
+----------+-----------+
|     Sales|      81000|
|   Finance|      79000|
| Marketing|      80000|
+----------+-----------+



In [16]:
# calculate avg salary in each department
df.groupBy("department").avg("salary").show()

+----------+-----------------+
|department|      avg(salary)|
+----------+-----------------+
|     Sales|85666.66666666667|
|   Finance|          87750.0|
| Marketing|          85500.0|
+----------+-----------------+



In [17]:
df.show()

+-------------+----------+-----+------+---+-----+
|employee_name|department|state|salary|age|bonus|
+-------------+----------+-----+------+---+-----+
|        James|     Sales|   NY| 90000| 34|10000|
|      Michael|     Sales|   NY| 86000| 56|20000|
|       Robert|     Sales|   CA| 81000| 30|23000|
|        Maria|   Finance|   CA| 90000| 24|23000|
|        Raman|   Finance|   CA| 99000| 40|24000|
|        Scott|   Finance|   NY| 83000| 36|19000|
|          Jen|   Finance|   NY| 79000| 53|15000|
|         Jeff| Marketing|   CA| 80000| 25|18000|
|        Kumar| Marketing|   NY| 91000| 50|21000|
+-------------+----------+-----+------+---+-----+



In [22]:
# calculate sum salary of department in states
df.groupBy("department", "state")\
	.sum("salary", "bonus") \
	.show()

+----------+-----+-----------+----------+
|department|state|sum(salary)|sum(bonus)|
+----------+-----+-----------+----------+
|   Finance|   NY|     162000|     34000|
| Marketing|   NY|      91000|     21000|
|     Sales|   CA|      81000|     23000|
| Marketing|   CA|      80000|     18000|
|   Finance|   CA|     189000|     47000|
|     Sales|   NY|     176000|     30000|
+----------+-----+-----------+----------+



In [23]:
df.groupBy("department", "state")\
	.avg("age", "bonus", "salary")\
	.show()

+----------+-----+--------+----------+-----------+
|department|state|avg(age)|avg(bonus)|avg(salary)|
+----------+-----+--------+----------+-----------+
|   Finance|   NY|    44.5|   17000.0|    81000.0|
| Marketing|   NY|    50.0|   21000.0|    91000.0|
|     Sales|   CA|    30.0|   23000.0|    81000.0|
| Marketing|   CA|    25.0|   18000.0|    80000.0|
|   Finance|   CA|    32.0|   23500.0|    94500.0|
|     Sales|   NY|    45.0|   15000.0|    88000.0|
+----------+-----+--------+----------+-----------+



In [32]:
# run mutiple aggreate function
df.groupBy("department").agg(
    sum("salary").alias("sum_salary"),\
    avg("salary").alias("avg_salary"),\
    sum("bonus").alias("sum_bonus")
).show()

+----------+----------+-----------------+---------+
|department|sum_salary|       avg_salary|sum_bonus|
+----------+----------+-----------------+---------+
|     Sales|    257000|85666.66666666667|    53000|
|   Finance|    351000|          87750.0|    81000|
| Marketing|    171000|          85500.0|    39000|
+----------+----------+-----------------+---------+



In [36]:
# use filter on aggreate function
df.groupBy("department").agg(
    sum("salary").alias("sum_salary"),\
    avg("salary").alias("avg_salary"),\
    sum("bonus").alias("sum_bonus")
) \
.filter(col("sum_bonus") > 50000) \
.show(truncate=False)

+----------+----------+-----------------+---------+
|department|sum_salary|avg_salary       |sum_bonus|
+----------+----------+-----------------+---------+
|Sales     |257000    |85666.66666666667|53000    |
|Finance   |351000    |87750.0          |81000    |
+----------+----------+-----------------+---------+



In [38]:
df.show()

+-------------+----------+-----+------+---+-----+
|employee_name|department|state|salary|age|bonus|
+-------------+----------+-----+------+---+-----+
|        James|     Sales|   NY| 90000| 34|10000|
|      Michael|     Sales|   NY| 86000| 56|20000|
|       Robert|     Sales|   CA| 81000| 30|23000|
|        Maria|   Finance|   CA| 90000| 24|23000|
|        Raman|   Finance|   CA| 99000| 40|24000|
|        Scott|   Finance|   NY| 83000| 36|19000|
|          Jen|   Finance|   NY| 79000| 53|15000|
|         Jeff| Marketing|   CA| 80000| 25|18000|
|        Kumar| Marketing|   NY| 91000| 50|21000|
+-------------+----------+-----+------+---+-----+



In [39]:
# use filter on aggreate function
df.groupBy("department").agg(
    sum("salary").alias("sum_salary"),\
    avg("salary").alias("avg_salary"),\
    sum("bonus").alias("sum_bonus")
) \
.where(col("sum_salary") >= 200000) \
.show(truncate=False)

+----------+----------+-----------------+---------+
|department|sum_salary|avg_salary       |sum_bonus|
+----------+----------+-----------------+---------+
|Sales     |257000    |85666.66666666667|53000    |
|Finance   |351000    |87750.0          |81000    |
+----------+----------+-----------------+---------+

